ContentBased v1_testing

Testiramo preciznost algoritma na prvih 100 korisnika. Svakom korisniku preprućamo 10 puta više filmova, nego
što ima u test_splitu filmova

In [1]:
import pandas as pd
import numpy as np
import itertools
from sklearn.model_selection import train_test_split

ratings = pd.read_csv("ml-latest-small/ratings.csv")
movies = pd.read_csv("ml-latest-small/movies.csv")

values = movies['genres'].str.split("|")

column_names = []
for v in values:
    if (len(column_names) == 20):
        break;
    for i in v:
        if i not in column_names:
            column_names.append(i)

column_names.append('title')
column_names.append('movieId')
df2 = pd.DataFrame(columns=column_names)

for index, row in movies.iterrows():
    tmp = row['genres'].split("|")
    df2.loc[index, 'title'] = movies.loc[index, 'title']
    df2.loc[index, 'movieId'] = movies.loc[index, 'movieId']
    for i in tmp:
        df2.loc[index, i] = 1
df2 = df2.fillna(0)
data = pd.merge(ratings, df2, on=['movieId'])
df2Helper = df2.drop(columns=['title', 'movieId'])

In [4]:
def findBestForUser(n):
    user = data.query("userId == {} & rating >= 3.5".format(n))
    if(len(user) < 5): return -1
    user_train, user_test = train_test_split(user, train_size=0.7, test_size=0.3, random_state=42)

    dictionary_train = {}
    for index, row in user_train.iterrows():
        dictionary_train[row['title']] = 1

    user_train = user_train.drop(columns=['title', 'userId', 'movieId', 'rating', 'timestamp'])
    user_train = np.array(user_train.mean())

    dictionary_test = {}
    for index, row in user_test.iterrows():
        dictionary_test[row['title']] = 1

    dictionary = {}
    for index, row in df2.iterrows():

        film = np.array(df2Helper.loc[index, :])
        result = user_train.dot(film) / (np.linalg.norm(user_train) * np.linalg.norm(film))
        if(df2.loc[index, 'title'] not in dictionary_train):
            dictionary[df2.loc[index, 'title']] = result

    sorted_dict = {}
    sorted_keys = sorted(dictionary, key=dictionary.get)
    sorted_keys = reversed(sorted_keys)
    for w in sorted_keys:
        sorted_dict[w] = dictionary[w]

    x = itertools.islice(sorted_dict.items(), 0, len(user_test) * 10)
    precision = 0
    for key, value in x:
        if(key in dictionary_test):
            precision += 1

    return (precision / (len(user_test) * 10))

precision = 0
counter = 0
for i in range(100):
    tmp = findBestForUser(i + 1)
    if(tmp != -1):
        precision += tmp
        counter += 1

print(precision / counter)

0.007581550834191959


Postigli smo preciznost od 0.7%.